In [1]:
import os
os.chdir('../..')
import numpy as np
import torch
import pickle
from tqdm import tqdm
from rdkit.Chem import MolFromInchi
from rdkit.Chem.rdmolops import Kekulize

RDKit WARNING: [07:33:15] Enabling RDKit 2019.09.3 jupyter extensions


In [2]:
from molgrad.net import MPNNPredictor

Using backend: pytorch


In [3]:
from molgrad.train import DEVICE

In [4]:
MODELS_PATH = "models/"

model_pt = os.path.join(MODELS_PATH, 'caco2_noHs.pt')

model = MPNNPredictor(node_in_feats=49,
                      edge_in_feats=10,
                      global_feats=4,
                      n_tasks=1).to(DEVICE) 
model.load_state_dict(torch.load(model_pt,
                                 map_location=DEVICE))

<All keys matched successfully>

In [10]:
from molgrad.prod import predict
from rdkit import Chem

In [17]:
mol = Chem.MolToInchi(Chem.MolFromSmiles("CCCCC"))

np.array(predict([mol, mol], w_path=model_pt)[:,0])

100%|██████████| 1/1 [00:00<00:00,  2.01it/s]


array([4.5073934, 4.5073934], dtype=float32)

In [18]:
from molgrad.vis import molecule_importance

In [19]:
svg, img, _, _, global_importance = molecule_importance(Chem.MolFromSmiles("CCC"),
                                                      model,
                                                      task=0,
                                                      version=2,
                                                      vis_factor=1,
                                                      eps=1e-3,
                                                      img_height=275,
                                                      img_width=550,
                                                      addHs=False)

In [20]:
svg

"<?xml version='1.0' encoding='iso-8859-1'?>\n<svg version='1.1' baseProfile='full'\n              xmlns='http://www.w3.org/2000/svg'\n                      xmlns:rdkit='http://www.rdkit.org/xml'\n                      xmlns:xlink='http://www.w3.org/1999/xlink'\n                  xml:space='preserve'\nwidth='550px' height='275px' viewBox='0 0 550 275'>\n<!-- END OF HEADER -->\n<rect style='opacity:1.0;fill:#FFFFFF;stroke:none' width='550' height='275' x='0' y='0'> </rect>\n<path d='M 25.7921,207.644 L 275.314,64.797' style='fill:none;fill-rule:evenodd;stroke:#FF0000;stroke-width:16px;stroke-linecap:butt;stroke-linejoin:miter;stroke-opacity:1' />\n<ellipse cx='25.7921' cy='207.644' rx='0.792144' ry='0.792144' style='fill:#FF0000;fill-rule:evenodd;stroke:#FF0000;stroke-width:1px;stroke-linecap:butt;stroke-linejoin:miter;stroke-opacity:1' />\n<ellipse cx='275.314' cy='64.797' rx='1.4802' ry='1.4802' style='fill:#FF0000;fill-rule:evenodd;stroke:#FF0000;stroke-width:1px;stroke-linecap:butt;

In [22]:
global_importance

array([ 0.5962544 ,  0.        , -0.00889703, -0.        ], dtype=float32)